# 如何为聊天机器人添加工具
:::info 前提条件
本指南假定您熟悉以下概念：
- [聊天机器人](/docs/concepts/messages)- [智能体](/docs/tutorials/agents)- [聊天记录](/docs/concepts/chat_history)
:::
本节将介绍如何创建对话代理：即能够使用工具与其他系统和API进行交互的聊天机器人。
:::note
本操作指南先前使用 [RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) 构建了一个聊天机器人。您可以在 [v0.2 文档](https://python.langchain.com/v0.2/docs/how_to/chatbots_tools/) 中访问该版本的指南。
截至 LangChain v0.3 版本发布，我们建议 LangChain 用户利用 [LangGraph 持久化功能](https://langchain-ai.github.io/langgraph/concepts/persistence/) 将 `memory` 集成到新的 LangChain 应用中。
如果你的代码已经在使用 `RunnableWithMessageHistory` 或 `BaseChatMessageHistory`，那么你**无需**做任何更改。我们近期没有计划弃用此功能，因为它适用于简单的聊天应用，任何使用 `RunnableWithMessageHistory` 的代码都将按预期继续运行。
请参阅[如何迁移到 LangGraph Memory](/docs/versions/migrating_memory/) 了解更多详情。:::
## 安装设置
在本指南中，我们将使用一个[工具调用代理](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/#tool-calling-agent)，该代理配备了一个用于网络搜索的工具。默认情况下，我们将使用[Tavily](/docs/integrations/tools/tavily_search)作为支持工具，但您可以将其替换为任何类似的工具。本节的其余部分将假设您正在使用Tavily。
您需要先在 [Tavily 网站注册账号](https://tavily.com/)，并安装以下软件包：

In [1]:
%pip install --upgrade --quiet langchain-community langchain-openai tavily-python langgraph

import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key:")

OpenAI API Key: ········
Tavily API Key: ········


您还需要将您的OpenAI密钥设置为`OPENAI_API_KEY`，并将您的Tavily API密钥设置为`TAVILY_API_KEY`。

## 创建智能体
我们的最终目标是创建一个能够以对话方式回应用户问题、并在需要时查找信息的智能代理。
首先，让我们初始化Tavily和一个支持工具调用的OpenAI[聊天模型](/docs/concepts/chat_models/)：

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

tools = [TavilySearchResults(max_results=1)]

# Choose the LLM that will drive the agent
# Only certain models support this
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

为了让我们的代理具备对话能力，我们还可以指定一个提示语。示例如下：

In [3]:
prompt = (
    "You are a helpful assistant. "
    "You may not need to use tools for every query - the user may just want to chat!"
)

太好了！现在让我们使用LangGraph预构建的[create_react_agent](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent)来组装我们的智能体，该功能允许你创建一个[工具调用型智能体](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/#tool-calling-agent)：

In [4]:
from langgraph.prebuilt import create_react_agent

# prompt allows you to preprocess the inputs to the model inside ReAct agent
# in this case, since we're passing a prompt string, we'll just always add a SystemMessage
# with this prompt string before any other messages sent to the model
agent = create_react_agent(model, tools, prompt=prompt)

## 运行代理程序
既然我们已经设置好了代理，现在就来尝试与它互动吧！它可以处理无需查询的简单问题：

In [5]:
from langchain_core.messages import HumanMessage

agent.invoke({"messages": [HumanMessage(content="I'm Nemo!")]})

{'messages': [HumanMessage(content="I'm Nemo!", additional_kwargs={}, response_metadata={}, id='39e715c7-bd1c-426f-8e14-c05586b3d221'),
  AIMessage(content='Hi Nemo! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 107, 'total_tokens': 118, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-6937c944-d702-40bb-9a9f-4141ddde9f78-0', usage_metadata={'input_tokens': 107, 'output_tokens': 11, 'total_tokens': 118})]}

或者，如果需要的话，它可以使用传递的搜索工具来获取最新信息：

In [6]:
agent.invoke(
    {
        "messages": [
            HumanMessage(
                content="What is the current conservation status of the Great Barrier Reef?"
            )
        ],
    }
)

{'messages': [HumanMessage(content='What is the current conservation status of the Great Barrier Reef?', additional_kwargs={}, response_metadata={}, id='a74cc581-8ad5-4401-b3a5-f028d69e4b21'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aKOItwvAb4DHQCwaasKphGHq', 'function': {'arguments': '{"query":"current conservation status of the Great Barrier Reef 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 116, 'total_tokens': 144, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-267ff8a8-d866-4ae5-9534-ad87ebbdc954-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current conservation status of the Great Barrier Reef 2023'}, 'id': 'call_aKOItwvAb4DHQCwaasKphGHq', 'type': 'tool_call'}], 

## 对话式回复
由于我们的提示中包含聊天历史消息的占位符，我们的智能体还能将先前的互动纳入考量，并像标准聊天机器人一样进行对话式回应：

In [7]:
from langchain_core.messages import AIMessage, HumanMessage

agent.invoke(
    {
        "messages": [
            HumanMessage(content="I'm Nemo!"),
            AIMessage(content="Hello Nemo! How can I assist you today?"),
            HumanMessage(content="What is my name?"),
        ],
    }
)

{'messages': [HumanMessage(content="I'm Nemo!", additional_kwargs={}, response_metadata={}, id='2c8e58bf-ad20-45a4-940b-84393c6b3a03'),
  AIMessage(content='Hello Nemo! How can I assist you today?', additional_kwargs={}, response_metadata={}, id='5e014114-7e9d-42c3-b63e-a662b3a49bef'),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='d92be4e1-6497-4037-9a9a-83d3e7b760d5'),
  AIMessage(content='Your name is Nemo!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 130, 'total_tokens': 136, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-17db96f8-8dbd-4f25-a80d-e4e872967641-0', usage_metadata={'input_tokens': 130, 'output_tokens': 6, 'total_tokens': 136})]}

如果愿意，您还可以为LangGraph代理添加记忆功能，以管理消息历史记录。让我们这样重新声明：

In [8]:
from langgraph.checkpoint.memory import MemorySaver

# highlight-start
memory = MemorySaver()
agent = create_react_agent(model, tools, prompt=prompt, checkpointer=memory)
# highlight-end

In [9]:
agent.invoke(
    {"messages": [HumanMessage("I'm Nemo!")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content="I'm Nemo!", additional_kwargs={}, response_metadata={}, id='117b2cfc-c6cc-449c-bba9-26fc545d0afa'),
  AIMessage(content='Hi Nemo! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 107, 'total_tokens': 118, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba16cc0b-fba1-4ec5-9d99-e010c3b702d0-0', usage_metadata={'input_tokens': 107, 'output_tokens': 11, 'total_tokens': 118})]}

然后如果我们重新运行封装后的代理执行器：

In [10]:
agent.invoke(
    {"messages": [HumanMessage("What is my name?")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content="I'm Nemo!", additional_kwargs={}, response_metadata={}, id='117b2cfc-c6cc-449c-bba9-26fc545d0afa'),
  AIMessage(content='Hi Nemo! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 107, 'total_tokens': 118, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba16cc0b-fba1-4ec5-9d99-e010c3b702d0-0', usage_metadata={'input_tokens': 107, 'output_tokens': 11, 'total_tokens': 118}),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='53ac8d34-99bb-43a7-9103-80e26b7ee6cc'),
  AIMessage(content='Your name is Nemo!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 130, 'total_tokens': 136, 'completion_tokens_details': {'reasoning_tok

这个 [LangSmith 追踪记录](https://smith.langchain.com/public/9e6b000d-08aa-4c5a-ac83-2fdf549523cb/r) 揭示了底层运行机制。
## 延伸阅读
要了解更多关于如何构建代理的信息，请查看这些 [LangGraph](https://langchain-ai.github.io/langgraph/) 指南：
* [智能体概念指南](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/)* [智能体教程](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/multi-agent-collaboration/)* [创建React代理](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/)
要了解更多关于工具使用的信息，您还可以查看[此用例部分](/docs/how_to#tools)。